In [1]:
#input:raw_test.xlsx
#output:processed_test.xlsx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
raw_test = pd.read_excel('raw_test.xlsx',index_col=0)

In [3]:
raw_test.head()

,admission number,Age,Sex,InhisTime,Revasc,Culp,MainVASc,Numsten,ECG,Killip,...,UA,Glu,WBC,N,HyperP,Cigaret,OtherHis,diabetes,Drugcom,MACE
358,1047495,45,男,2019-09-25,1h48min,4.0,3.0,1.0,1,I,...,117.0,5.30,6.52,4.54,0,1,1,1,1.0,0
19,899915,67,男,2019-01-12,18h30min,2.0,2.0,3.0,1,III,...,718.0,6.93,11.97,10.79,1,1,1,0,1.0,0
294,1065946,50,男,2019-12-22,9h,3.0,4.0,NaN,0,I,...,NaN,5.69,NaN,NaN,0,1,0,0,1.0,0
194,869417,72,男,2018-11-25,7d48min,3.0,3.0,1.0,0,II,...,495.0,8.71,9.60,7.79,1,0,1,0,1.0,0
117,896240,59,男,2019-01-07,5h30min,NaN,NaN,2.0,0,Ⅰ,...,370.0,9.24,10.02,5.43,0,0,0,0,1.0,0


In [4]:
#Encode gender(man equal to 0,women equal to 0)
raw_test.loc[raw_test['Sex'] == '男', 'Sex'] = 0
raw_test.loc[raw_test['Sex'] == '女', 'Sex'] = 1

In [5]:
#Calculate the number of days after discharge(follow-up time = follow-up data - hospital date)
import datetime
import time
follow_date = datetime.datetime(2020,11,1,0,0,0)
ts = pd.DatetimeIndex([follow_date])
for i in raw_test.index:
    a = ts - raw_test['InhisTime'][i]
    b = a.days
    raw_test['InhisTime'][i] = b[0]        
raw_test.rename(columns={'InhisTime':'Follow_up time'},inplace=True)

C:\Users\lenovo\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [6]:
raw_test.head()

,admission number,Age,Sex,Follow_up time,Revasc,Culp,MainVASc,Numsten,ECG,Killip,...,UA,Glu,WBC,N,HyperP,Cigaret,OtherHis,diabetes,Drugcom,MACE
358,1047495,45,0,403,1h48min,4.0,3.0,1.0,1,I,...,117.0,5.30,6.52,4.54,0,1,1,1,1.0,0
19,899915,67,0,659,18h30min,2.0,2.0,3.0,1,III,...,718.0,6.93,11.97,10.79,1,1,1,0,1.0,0
294,1065946,50,0,315,9h,3.0,4.0,NaN,0,I,...,NaN,5.69,NaN,NaN,0,1,0,0,1.0,0
194,869417,72,0,707,7d48min,3.0,3.0,1.0,0,II,...,495.0,8.71,9.60,7.79,1,0,1,0,1.0,0
117,896240,59,0,664,5h30min,NaN,NaN,2.0,0,Ⅰ,...,370.0,9.24,10.02,5.43,0,0,0,0,1.0,0


In [7]:
raw_test.isnull().sum()

admission number     0
Age                  0
Sex                  0
Follow_up time       0
Revasc               0
Culp                 2
MainVASc             6
Numsten              1
ECG                  0
Killip               0
TC                   8
LDL                  8
CRP                 10
EF                   7
LV                   7
Cr                  11
UA                  11
Glu                  7
WBC                 16
N                   17
HyperP               0
Cigaret              0
OtherHis             0
diabetes             0
Drugcom              2
MACE                 0
dtype: int64

In [8]:
'''
input:a discrete variable
output:one of all discrete variable categories
function:generating one of all discrete variable categories according to the proportion of this categort
For example,the number of man(labelled as 0) is 144,the number of woman(labelled as 0) is 100,
then this function has a probability of 144/(144+100) for producing the value 0
'''
def randnum(data):
    a = np.random.random()
    count_values = (data.value_counts()).values
    count_index = (data.value_counts()).index
    notnull_datalen = data.notnull().sum()
    b = []
    sum_value = 0
    return_value = []
    for i in range(len(data.value_counts())):
        sum_value += count_values[i]
        b.append(sum_value/notnull_datalen)
        if a < b[i]:
            return_value = count_index[i]
            break
    return return_value

In [9]:
#Drugcom imputation
Drugcom = raw_test['Drugcom']
Drugcom_is_null = pd.isnull(Drugcom)
Drugcom_null_true = Drugcom[Drugcom_is_null]
for index_null in Drugcom_null_true.index:
    raw_test['Drugcom'][index_null] = randnum(raw_test['Drugcom'])

C:\Users\lenovo\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [10]:
#Culp imputation
Culp = raw_test['Culp']
Culp_is_null = pd.isnull(Culp)
Culp_null_true = Culp[Culp_is_null]
for index_null in Culp_null_true.index:
    raw_test['Culp'][index_null] = randnum(raw_test['Culp'])

C:\Users\lenovo\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [11]:
#MainVASc imputation
MainVASc = raw_test['MainVASc']
MainVASc_is_null = pd.isnull(MainVASc)
MainVASc_null_true = MainVASc[MainVASc_is_null]
for index_null in MainVASc_null_true.index:
    raw_test['MainVASc'][index_null] = randnum(raw_test['MainVASc'])

C:\Users\lenovo\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [12]:
#Numsten imputation
Numsten = raw_test['Numsten']
Numsten_is_null = pd.isnull(Numsten)
Numsten_null_true = Numsten[Numsten_is_null]
raw_test['Numsten'][Numsten_null_true.index] = randnum(raw_test['Numsten'])

C:\Users\lenovo\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [13]:
#The imputation of discrete variables is complete
raw_test.isnull().sum()

admission number     0
Age                  0
Sex                  0
Follow_up time       0
Revasc               0
Culp                 0
MainVASc             0
Numsten              0
ECG                  0
Killip               0
TC                   8
LDL                  8
CRP                 10
EF                   7
LV                   7
Cr                  11
UA                  11
Glu                  7
WBC                 16
N                   17
HyperP               0
Cigaret              0
OtherHis             0
diabetes             0
Drugcom              0
MACE                 0
dtype: int64

In [14]:
'''
input:a continuous variable
output:None
function:imputing a continuous variable with the values generated by Gaussian distribution.
if the generated values are not in the range of the continuous variable,the variable will be imputed with the mean of this variable
'''
def fill_null(data):
    data_is_null = pd.isnull(data)
    data_null_true = data[data_is_null]
    for index_null in data_null_true.index:
        gauss = np.random.normal(np.mean(data),np.var(data),1)
        if gauss > raw_test[data.name].max():
            raw_test[data.name][index_null] = np.mean(data)
        elif gauss < raw_test[data.name].min():
            raw_test[data.name][index_null] = np.mean(data)
        else:
            raw_test[data.name][index_null] = gauss

In [15]:
postive_train = raw_test[raw_test['MACE'] == 1][:]
negative_train = raw_test[raw_test['MACE'] == 0][:]

In [16]:
#imputation of TC
postive_TC = postive_train['TC']
negative_TC = negative_train['TC']
fill_null(postive_TC)
fill_null(negative_TC)

C:\Users\lenovo\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\lenovo\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [17]:
#imputation of LDL
postive_LDL = postive_train['LDL']
negative_LDL = negative_train['LDL']
fill_null(postive_LDL)
fill_null(negative_LDL)

C:\Users\lenovo\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [18]:
#imputation of CRP
postive_CRP = postive_train['CRP']
negative_CRP = negative_train['CRP']
fill_null(postive_CRP)
fill_null(negative_CRP)

C:\Users\lenovo\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\Users\lenovo\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\lenovo\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import k

In [19]:
#imputation of EF
postive_EF = postive_train['EF']
negative_EF = negative_train['EF']
fill_null(postive_EF)
fill_null(negative_EF)

C:\Users\lenovo\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [20]:
#imputation of LV
postive_LV = postive_train['LV']
negative_LV = negative_train['LV']
fill_null(postive_LV)
fill_null(negative_LV)

C:\Users\lenovo\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
C:\Users\lenovo\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\lenovo\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  

In [21]:
#imputation of Cr
postive_Cr = postive_train['Cr']
negative_Cr = negative_train['Cr']
fill_null(postive_Cr)
fill_null(negative_Cr)

C:\Users\lenovo\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
C:\Users\lenovo\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\Users\lenovo\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vie

In [22]:
#imputation of UA
postive_UA = postive_train['UA']
negative_UA = negative_train['UA']
fill_null(postive_UA)
fill_null(negative_UA)

C:\Users\lenovo\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\Users\lenovo\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [23]:
#imputation of Glu
postive_Glu = postive_train['Glu']
negative_Glu = negative_train['Glu']
fill_null(postive_Glu)
fill_null(negative_Glu)

C:\Users\lenovo\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
C:\Users\lenovo\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\lenovo\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  

In [24]:
#imputation of WBC
postive_WBC = postive_train['WBC']
negative_WBC = negative_train['WBC']
fill_null(postive_WBC)
fill_null(negative_WBC)

C:\Users\lenovo\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\lenovo\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\Users\lenovo\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import k

In [25]:
#imputation of N
postive_N = postive_train['N']
negative_N = negative_train['N']
fill_null(postive_N)
fill_null(negative_N)

C:\Users\lenovo\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
C:\Users\lenovo\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\Users\lenovo\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vie

In [26]:
#The imputation of continuous and discrete variables is complete
raw_test.isnull().sum()

admission number    0
Age                 0
Sex                 0
Follow_up time      0
Revasc              0
Culp                0
MainVASc            0
Numsten             0
ECG                 0
Killip              0
TC                  0
LDL                 0
CRP                 0
EF                  0
LV                  0
Cr                  0
UA                  0
Glu                 0
WBC                 0
N                   0
HyperP              0
Cigaret             0
OtherHis            0
diabetes            0
Drugcom             0
MACE                0
dtype: int64

In [27]:
#input：strings of 'Revasc(Time of revascularization)'
#output：minutes
#d,h,m convert to minutes
def time_conv(strings):
    day = 0
    hour = 0
    mins = 0
    if 'd' in strings:
        d_end = strings.index('d')
        day = int(strings[0:d_end])
    if 'h' in strings:
        if 'd' in strings:
            h_star = strings.index('d')+1
            h_end = strings.index('h')
            hour = int(strings[h_star:h_end])
        else:
            h_end = strings.index('h')
            hour = int(strings[0:h_end])
    if 'm' in strings:
        if 'h' in strings:
            m_star = strings.index('h')+1
            m_end = strings.index('m')
            mins = int(strings[m_star:m_end])
        elif 'd' in strings:
            m_star = strings.index('d')+1
            m_end = strings.index('m')
            mins = int(strings[m_star:m_end])
        else:
            m_end = strings.index('m')
            mins = int(strings[0:m_end])
    if day > 3:
        day = 3
    time_mins = day*1440+hour*60+mins
    return time_mins

In [28]:
for i in raw_test.index:
    conv = raw_test['Revasc'][i] 
    raw_test['Revasc'][i] = time_conv(conv)
raw_test['Revasc'] = raw_test['Revasc'].astype(int)

C:\Users\lenovo\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [29]:
raw_test.head()

,admission number,Age,Sex,Follow_up time,Revasc,Culp,MainVASc,Numsten,ECG,Killip,...,UA,Glu,WBC,N,HyperP,Cigaret,OtherHis,diabetes,Drugcom,MACE
358,1047495,45,0,403,108,4.0,3.0,1.0,1,I,...,117.000000,5.30,6.520000,4.540000,0,1,1,1,1.0,0
19,899915,67,0,659,1110,2.0,2.0,3.0,1,III,...,718.000000,6.93,11.970000,10.790000,1,1,1,0,1.0,0
294,1065946,50,0,315,540,3.0,4.0,0.0,0,I,...,336.266667,5.69,9.946667,22.388009,0,1,0,0,1.0,0
194,869417,72,0,707,4368,3.0,3.0,1.0,0,II,...,495.000000,8.71,9.600000,7.790000,1,0,1,0,1.0,0
117,896240,59,0,664,330,4.0,3.0,2.0,0,Ⅰ,...,370.000000,9.24,10.020000,5.430000,0,0,0,0,1.0,0


In [30]:
for i in raw_test.index:
    if raw_test['Killip'][i] == 'I':
        raw_test['Killip'][i] = 1
    elif raw_test['Killip'][i] == 'Ⅰ':
        raw_test['Killip'][i] = 1
    elif raw_test['Killip'][i] == 'I ':
        raw_test['Killip'][i] = 1
    elif raw_test['Killip'][i] == 'II':
        raw_test['Killip'][i] = 2
    elif raw_test['Killip'][i] == 'Ⅱ':
        raw_test['Killip'][i] = 2
    elif raw_test['Killip'][i] == 'III':
        raw_test['Killip'][i] = 3
    elif raw_test['Killip'][i] == 'Ⅲ':
        raw_test['Killip'][i] = 3
    elif raw_test['Killip'][i] == 'IV':
        raw_test['Killip'][i] = 4
    elif raw_test['Killip'][i] == 'Ⅳ':
        raw_test['Killip'][i] = 4

C:\Users\lenovo\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\lenovo\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\Users\lenovo\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/u

In [31]:
raw_test['Killip'] = raw_test['Killip'].astype(int)

In [32]:
raw_test['Sex'] = raw_test['Sex'].astype(int)
raw_test['Follow_up time'] = raw_test['Follow_up time'].astype(int)

In [33]:
raw_test = raw_test.astype(float)

In [34]:
raw_test.to_excel(r'C:\Users\lenovo\Revision\Preprocessing\processed_test.xlsx')

In [35]:
raw_test.dtypes

admission number    float64
Age                 float64
Sex                 float64
Follow_up time      float64
Revasc              float64
Culp                float64
MainVASc            float64
Numsten             float64
ECG                 float64
Killip              float64
TC                  float64
LDL                 float64
CRP                 float64
EF                  float64
LV                  float64
Cr                  float64
UA                  float64
Glu                 float64
WBC                 float64
N                   float64
HyperP              float64
Cigaret             float64
OtherHis            float64
diabetes            float64
Drugcom             float64
MACE                float64
dtype: object

In [36]:
raw_test.isnull().sum()

admission number    0
Age                 0
Sex                 0
Follow_up time      0
Revasc              0
Culp                0
MainVASc            0
Numsten             0
ECG                 0
Killip              0
TC                  0
LDL                 0
CRP                 0
EF                  0
LV                  0
Cr                  0
UA                  0
Glu                 0
WBC                 0
N                   0
HyperP              0
Cigaret             0
OtherHis            0
diabetes            0
Drugcom             0
MACE                0
dtype: int64